<a href="https://colab.research.google.com/github/nmarcella/AI_projects/blob/main/Camera_dog_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup OLLAMA



*   After running GPU_server.ipynb, copy ngrok link to the url variable. note the /api/generate for querying the model and /api/pull for installing the desired model.



In [ ]:
url = 'https://f398-34-82-56-162.ngrok-free.app' # this link is no longer valid
url_gen = url + '/api/generate'
url_pull = url + '/api/pull'

In [ ]:
!curl url_pull -d '{"model": "llava"}'

# The main loop



*   javascript is used to access the camera feed. (accept permissions)
*   an image is saved to the file system
*   the image is resized to fit the llava requirements
*   the image is encoded to bit64
*   llava is queried
*   The resonse is recorded.



In [ ]:
import requests
import json
import base64
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import time
import glob
import os

headers = {
    'Content-Type': 'application/json',
}

url = url_gen  # Replace with your actual API endpoint

def generate_response(prompt, image_path):
    with open(image_path, 'rb') as f:
        img_str = base64.b64encode(f.read()).decode('utf-8')

    CONTEXT = "you are an AI assistant tasked with keeping tabs on my dogs. Please look the image and respond with a simple yes or no. Do not respond with any other text besides 'yes' or 'no'. The images may be blurry, use your best judgment."

    data = {
        "model": "llava",  # Replace with your actual model name if different
        "stream": False,
        "prompt": "system prompt: " + CONTEXT + " human question: "+ prompt,
        "images": [img_str],
    }

    response = requests.post(url, headers=headers, data=json.dumps(data))
    if response.status_code == 200:
        response_data = json.loads(response.text)
        return response_data["response"]
    else:
        print("Error:", response.status_code, response.text)
        return None

def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Capture the photo after 1000 ms
            await new Promise(resolve => setTimeout(resolve, 1000));

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getVideoTracks()[0].stop();
            video.remove();
            return canvas.toDataURL('image/jpeg', quality);
        }
        ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return filename

def resize_image(image_path, new_size=(672, 672)): # apparently the ideal size is 672 672 for LLaVA but not sure how much this matters
    image = Image.open(image_path)
    resized_image = image.resize(new_size)
    resized_image.save('prepro.jpg')
    return 'prepro.jpg'

# Log variable to store descriptions
log = []

try:
    # Loop to run indefinitely
    while True:
        photo_filename = take_photo()
        resized_photo = resize_image(photo_filename)
        description = generate_response("Is there a dog on the couch?", resized_photo)
        log.append(description)
        print(description)
        time.sleep(10)  # Pause for 30 seconds before the next photo

except KeyboardInterrupt:
    print("Interrupted by user, cleaning up...")
    # Delete images created during the loop
    for filename in glob.glob('photo.jpg') + glob.glob('prepro.jpg'):
        os.remove(filename)
    print("Images cleaned up successfully.")
